# The Asymmetric 2D Pendulum and Lissajous Curves

*By* Timothy A. V. Teatro 

A presentation for UOIT's annual Science Rendezvous

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import scipy.integrate as integrate
%pylab

To model a 2D-pendulum, I'll divide it into two single-dimensional pendulums. Since we only need to work on one dimension at a time, let us consider Newton's equation of motion for a pendulum with drag in a single dimension:

$$
\ddot\theta - \frac{g}{m\,r}\,\sin\theta + \frac{b}{m}\,\dot{\theta} = 0.
$$

The $\sin\theta$ term is the restoration term. It is the balance of tension and gravity coercing the pendulum pith back towards the resting position. The $\dot{\theta}$ term is the drag term. It represents parasitic losses from the air drag on the ball, and friction in the hinges. Notice that it is proportional to velocity: the faster things move, the faster energy is lost to the environment.

It is useful to represent the system as a vector valued function on a two-dimensional state space $\boldsymbol{x}\in\mathbf{R}$ where the first dimension is the angular displacement, $\theta$, and the second dimension is the angular speed $\dot{\theta} = \omega$. The vector equation for the system is

$$
\dot{\boldsymbol{x}} = \frac{\mathrm{d}}{\mathrm{d}t}\begin{bmatrix}\theta\\ \dot\theta\end{bmatrix} = \begin{bmatrix}\dot\theta\\ \frac{-g}{m\,r}\sin\theta + \frac{b}{m}\,\dot{\theta}\end{bmatrix} =: \boldsymbol{f}(\boldsymbol{x}),
$$

The energy of the pendulum will by the sum of Kinetic energy:

$$
  T = \frac{1}{2}\,m\,(r\,\dot{\theta})^2,
$$

and potential energy:

$$
  U = m\,g\,r\,\cos\theta,
$$

so $E = T + U$.

In Python, we'll express all of this in a class containing no state except the parameters

In [ ]:
class Pendulum1D:
    """ Just the dynamics of the pendulum. Constants related to pendulum
        design are stored here, but not the pendulum's state.
    """

    def __init__(self, r=1, m=.5, b=.2, g=9.807):
        self.r = r  # Radius, or length of string.
        self.m = m  # Pith mass
        self.b = b  # Drag loss coefficient
        self.g = g  # Acceleration due to gravity

    def dxdt(self, x, t):
        # State of pendulum: x ∈ R^2, contains angular position and speed.
        return np.array([x[1], -9.8 * np.sin(x[0]) / self.r - self.b * x[1]])

    def energy(s, x):
        T = s.m * (s.r * x[1])**2 / 2.
        U = -s.m * s.g * s.r * np.cos(x[0])
        return T + U


The state of the simulation is contained in an object type, which aggregates the two pendulums. It also has a method for computing the total energy of the pendulums, and a utility for converting angular to Cartesian coordinates.

In [ ]:
class PendulumSimulator2D:
    """ The class aggregates dynamics for two 1D pendulums which is isomorphic
        do a single 2D pendulum. (The x-direction doesn't care what the
        y-direction is doing, so the 2D problem is trivially decoupled into
        two 1D problems.)
    """

    # yapf: disable
    def __init__(self, r=[1, 1], m=0.5, b=0.2, g=9.807, dt=0.1,  # As before
                 xx0=[0, 1],    # Initial state of x-pendulum
                 xy0=[0, -1]):  # Initial state of y-pendulum
        self.dt = dt
        self.px = Pendulum1D(r=r[0], m=m, b=b, g=g)
        self.py = Pendulum1D(r=r[1], m=m, b=b, g=g)

        # Initial conditions
        self.t = 0
        self.xx = np.array(xx0)
        self.xy = np.array(xy0)
        self.E0 = self.energy(xx0, xy0)  # Starting energy
        self.Erest = self.energy([0, 0], [0, 0])  # Rest energy
    # yapf: enable

    def energy(self, xx, xy):
        return self.px.energy(xx) + self.py.energy(xy)

    def angular_to_cartesian(self, pX):
        """ The dynamics are performed on the angular coordinates. For plotting,
            we'll want to convert with some basic trigonometry.
                x = r * sin(theta)
                dx/dt = r * cos(theta) * d.theta/dt
        """
        return np.array(
            [self.px.r * np.sin(pX[0]), self.py.r * np.cos(pX[0]) * pX[1]])

    def __iter__(self):
        return self

    def __next__(self):
        # Will return current data, and tee-up for next iteration.
        prevX = (self.t, self.angular_to_cartesian(self.xx),
                 self.angular_to_cartesian(self.xy))
        self.xx = integrate.odeint(self.px.dxdt, self.xx, [0, self.dt])[-1]
        self.xy = integrate.odeint(self.py.dxdt, self.xy, [0, self.dt])[-1]
        self.t += self.dt
        return prevX

I recommend running the Python files. For some reason, the performance of the following block suffers in Jupyter.

In [ ]:
    # Everything to do with plotting and animating.
    preset = []
    preset.append(dict(r=[2, 1.4], b=0.01))
    preset.append(dict(r=[2, 1.8], b=0.02))
    sim = PendulumSimulator2D(**preset[1], dt=0.03)

    fig = plt.figure()
    ax = fig.add_subplot(
        111, aspect='equal', autoscale_on=False, xlim=(-1, 1), ylim=(-1, 1))
    fig.subplots_adjust(
        left=0.05, right=0.98, bottom=0.05, top=0.98, wspace=0.1)

    line, = ax.plot([], [], 'r-', lw=1)
    pith, = ax.plot([], [], 'o-', ms=15)
    time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

    XHistory = []
    YHistory = []

    def animate(data):
        t, X, Y = data
        XHistory.append(X[0])
        YHistory.append(Y[0])
        pith.set_data(X[0], Y[0])
        line.set_data(XHistory, YHistory)
        time_text.set_text('time = %.1f' % t)
        return line, pith, time_text

    ani = animation.FuncAnimation(fig, animate, sim, blit=True, interval=10)
    plt.show()

Copyright (C) 2017 By Timothy A. V. Teatro. All rights reserved.